In [ ]:
#!curl -L https://d36yt3mvayqw5m.cloudfront.net/public/nuimages-v1.0/nuimages-v1.0-all-metadata.tgz > data_annotation.tgz # real annotation

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  609M  100  609M    0     0   100M      0  0:00:06  0:00:06 --:--:--  107M


In [ ]:
#!curl -L https://d36yt3mvayqw5m.cloudfront.net/public/nuimages-v1.0/nuimages-v1.0-all-samples.tgz > data_image.tgz # real images

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.2G  100 15.2G    0     0  14.4M      0  0:17:57  0:17:57 --:--:-- 12.4M


In [ ]:
# minified version just for testing
#!curl -L https://d36yt3mvayqw5m.cloudfront.net/public/nuimages-v1.0/nuimages-v1.0-mini.tgz > mini.tgz && tar -xzf mini.tgz && mv v1.0-mini v1.0-train

In [ ]:
from google.colab import drive # the real images data files take too long (about 17 minutes) to be downloaded, so we just mount it
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xzf /content/drive/MyDrive/nuimages/nuimages-v1.0-all-metadata.tgz && tar -xzf /content/drive/MyDrive/nuimages/nuimages-v1.0-all-samples.tgz # extract. this will take about 7 minutes

In [ ]:
drive.flush_and_unmount() # unmount when we don't need it anymore, so we have more disk space

In [ ]:
!mkdir train test val && mkdir train/images test/images val/images train/labels test/labels val/labels # for the yolo format

In [ ]:
import pandas as pd

def bbox_to_yolo(xmin, ymin, xmax, ymax, image_width, image_height):
    box_width = xmax - xmin
    box_height = ymax - ymin

    x_center = xmin + box_width / 2
    y_center = ymin + box_height / 2

    # Normalize by image size
    x_center_norm = x_center / image_width
    y_center_norm = y_center / image_height
    width_norm = box_width / image_width
    height_norm = box_height / image_height

    return x_center_norm, y_center_norm, width_norm, height_norm


def read_and_join(path_prefix, percent):
    # percent: take only certain percent of the images
    obj_ann = pd.read_json(path_prefix + 'object_ann.json')
    category = pd.read_json(path_prefix + 'category.json').rename(columns={"name": "category_name"})
    category = category.sort_values(by='category_name')
    category['category_id'] = range(len(category))

    sample_data = pd.read_json(path_prefix + 'sample_data.json')
    sample_data = sample_data[sample_data['filename'].str.startswith('sample')]
    n_rows = int(len(sample_data) * (percent / 100))
    sample_data = sample_data.iloc[:n_rows]

    # attribute = pd.read_json(path_prefix + 'attribute.json')

    # object annotation inner join sample_data
    merged = obj_ann.merge(
        sample_data,
        how = 'inner',
        right_on='token',
        left_on='sample_data_token',
        suffixes=('_left', '_right')
    )

    merged = merged.merge(
        category,
        how = 'left',
        left_on='category_token',
        right_on='token',
        suffixes=('_from_merged', '_from_category')
    )

    print (len(obj_ann))
    print (len(sample_data))
    print (len(category))
    print ("merged: ", len(merged))

    return merged

# create list of train/val/test image file name list in a txt file (train/val.txt), so we can use cp/rsync with it
def create_image_filename_list(path_prefix, data_split, merged):
    if data_split not in ['train', 'val', 'test']:
        return -1

    df = merged[['filename']]
    filenames = set([])
    for index, row in df.iterrows():
        filenames.add(row['filename'])
    for fn in filenames:
        with open(data_split+".txt", 'a') as f:
            f.write(fn + "\n")

# create yolo label txt
def convert_to_yolo_format(path_prefix, data_split, merged):
    if data_split not in ['train', 'val', 'test']:
        return -1
    df = merged[['bbox', 'category_id', 'category_name', 'filename', 'width',  'height']]
    for index, row in df.iterrows():
        filename = row['filename'].split("/")[-1] # filename column is in the format of samples/CAM_BACK/n010-2018-08-27-16-15-24+0800...
        abs_path_to_label_txt = path_prefix + data_split + "/labels/" + filename[:len(filename)-4] + '.txt' # example: /content/train/label/asdf.txt. remove the .jpg from asdf.jpg first
        x_center, y_center, width, height = bbox_to_yolo(row['bbox'][0], row['bbox'][1], row['bbox'][2], row['bbox'][3], row['width'], row['height'])
        with open(abs_path_to_label_txt, 'a') as f: # append label to the txt
            f.write(str(row['category_id']) + " " + str(x_center) + " " + str(y_center) + " " + str(width) + " " + str(height) + "\n")

path_prefix = '/content/v1.0-val.'
val_df = read_and_join('/content/v1.0-val/', 1) # only take 1% just for testing
train_df = read_and_join('/content/v1.0-train/', 1)

136074
164
25
merged:  1534
557715
672
25
merged:  5406


In [ ]:
# NOTE: do NOT run convert_to_yolo_format to mounted Drive directory, it's slow
create_image_filename_list('/content/', 'train', train_df)
create_image_filename_list('/content/', 'val', val_df)
convert_to_yolo_format('/content/', 'train', train_df)
convert_to_yolo_format('/content/', 'val', val_df)

In [ ]:
# make sure the val.txt exists before running this
# copy the val image to directory expected by yolo. should take about 1 minute for the entire dataset
!xargs -a val.txt -P8 -I{} cp "/content/{}" /content/val/images/

In [ ]:
# make sure the train.txt exists before running this
# copy the train image to directory expected by yolo. should take about 5 minutes for the entire dataset
!xargs -a train.txt -P8 -I{} cp "/content/{}" /content/train/images/

In [ ]:
train_df['filename'].head()

,filename
0,samples/CAM_BACK_RIGHT/n009-2018-05-08-15-52-4...
1,samples/CAM_BACK/n016-2018-09-10-11-01-25+0800...
2,samples/CAM_BACK_LEFT/n016-2018-07-02-11-25-51...
3,samples/CAM_FRONT_RIGHT/n015-2018-09-05-11-47-...
4,samples/CAM_FRONT_LEFT/n008-2018-06-04-16-30-0...


In [ ]:
!ls -1 /content/val/images/ | wc -l

148


In [ ]:
!ls -1 /content/val/labels/ | wc -l

148


In [ ]:
import yaml

class InlineList(list): pass

def represent_inline_list(dumper, data):
    return dumper.represent_sequence('tag:yaml.org,2002:seq', data, flow_style=True)

def create_data_yml(path_prefix):
  category = pd.read_json(path_prefix + 'category.json').rename(columns={"name": "category_name"})
  category = category.sort_values(by='category_name')
  category['category_id'] = range(len(category))

  yml_data = {
      'train': '../train/images',
      'val': '../val/images',
      "nc": len(category),
      "names": InlineList([row['category_name'] for index, row in category.iterrows()])
  }
  yaml.add_representer(InlineList, represent_inline_list)
  with open('/content/data.yml', 'w') as outfile:
    yaml.dump(yml_data, outfile,default_flow_style=False, sort_keys=False)


create_data_yml('/content/v1.0-test/')

In [ ]:
!pip install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
!yolo detect train model=yolov8n.pt data=data.yml epochs=50 imgsz=640 rect=True batch=16 project=coba

Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=Fals

In [ ]:
from ultralytics import YOLO

# 1) Load the fine-tuned weights
model = YOLO("/content/coba/train3/weights/best.pt")

model.predict(
    source="/content/samples/CAM_FRONT_RIGHT/n003-2018-01-02-11-48-43+0800__CAM_FRONT_RIGHT__1514865473348977.jpg",  # or a local path / folder
    conf=0.25,
    save=True,        # annotated image saved to runs/detect/predict/
    imgsz=640
)
print("Prediction saved to runs/detect/predict/")



image 1/1 /content/samples/CAM_FRONT_RIGHT/n003-2018-01-02-11-48-43+0800__CAM_FRONT_RIGHT__1514865473348977.jpg: 384x640 1 vehicle.car, 16.5ms
Speed: 4.6ms preprocess, 16.5ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict2
Prediction saved to runs/detect/predict/
